In [1]:
# from tensorflow.keras.appliications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import AUC,Precision,Recall
import tensorflow as tf
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Tensorboard

from tensorflow.keras.mixed_precision import set_global_policy
import numpy as np
import os

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)

2025-04-29 11:09:00.367168: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-29 11:09:00.583406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745896140.683110    1232 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745896140.705591    1232 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-29 11:09:01.049089: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# # https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images
# base_model = EfficientNetB0(include_top=False,weights = 'imagenet')

# inputs = layers.Input(shape=(224,224,1))
# x = layers.Conv2D(3,(3,3), padding = 'same')(inputs)
# x = base_model(x)
# x = layers.GlobalAveragePooling2D()(x)
# outputs = layers.Dense(8,activation='sigmoid')(x)

# model = models.Model(inputs,outputs)
# model.summary()

I0000 00:00:1745867673.718098    1193 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-04-29 03:14:33.923846: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_1 (Cast)                   │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │        10,248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,059,849 (15.49 MB)

 Trainable params: 4,017,826 (15.33 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [2]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Use mixed precision
# https://keras.io/api/mixed_precision/
set_global_policy('mixed_float16')

# Clear GPU memory
tf.keras.backend.clear_session()


with tf.device('/GPU:0'):
    base_model = EfficientNetB0(include_top=False,weights = 'imagenet')

    inputs = layers.Input(shape=(224,224,1))
    x = layers.Conv2D(3,(3,3), padding = 'same')(inputs)
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(8,activation='sigmoid')(x)

    model = models.Model(inputs,outputs)

model.summary()


I0000 00:00:1745896166.228366    1232 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1745896166.231146    1232 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │        10,248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,059,849 (15.49 MB)

 Trainable params: 4,017,826 (15.33 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [3]:
model.compile(optimizer="adam",loss="binary_crossentropy", metrics = ["binary_accuracy", "val_accuracy"
                                                                    #   ,AUC(), Precision(),Recall()
                                                                      ])


In [4]:

X_train = np.load("../data/numpy/X_train.npy")
Y_train = np.load("../data/numpy/Y_train.npy")

X_val = np.load("../data/numpy/X_valid.npy")
Y_val = np.load("../data/numpy/Y_valid.npy")



In [ ]:
# my_callbacks = []

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val))

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    batch_size = 8,
    verbose = 1
)


Epoch 1/10


: 

In [ ]:
# history = model.fit(
#     train_ds,
#     validation_data = (X_val, Y_val),
#     epochs = 10,
#     batch_size = 8,
#     verbose = 1
#     )

2025-04-29 03:02:25.117474: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.35GiB (rounded to 2526662656)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-04-29 03:02:25.128615: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for GPU_0_bfc
2025-04-29 03:02:25.134211: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (256): 	Total Chunks: 107, Chunks in use: 107. 26.8KiB allocated for chunks. 26.8KiB in use in bin. 5.4KiB client-requested in use in bin.
2025-04-29 03:02:25.135135: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (512): 	Total Chunks: 64, Chunks in use: 64. 39.8KiB allocated for chunks. 39.8KiB in use in bin. 32.6KiB client-requested in use in bin.


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["binary_accuracy"])
plt.plot(history.history["val_accuracy"])
plt.show()